<h1>Homework 5: Understanding linear and trees-based models</h1>
<h2>Predictive Analytics using Python (CIS432)</h2>
<h3>Simon Business School</h3>

__Instructor__: Yaron Shaposhnik




### Instructions

In this homework assignment, you will train linear and tree-based models. You will experiment with their key hyper-parameters and observe how they affect the resulting models. This should improve your understanding of these models and help you create better ones.


### Answer key

This assignment (as well as others) is graded by comparing your answers (that is, the variables and Python objects you create) with precomputed answers. This allows you to get immediate feedback in order to find your errors and correct them. The downside of this approach is that the grading code is strict and even slight deviations from the desired outputs could result in reduction of points. 

To make this learning experience more efficient, the objects that you are asked to generate are provided to you in the variable `ANSWER_KEY`. Questions may ask you to assign some value (like a number or object such as data frame) to some variable. 
For example, you might be asked to assign the variable `n_rows` with some value. To view the correct answer simply run the command `ANSWER_KEY['n_rows']`. 

Note that the answer key is provided to you __for debugging purposes only__. Using it in your final submission or hard-coding solutions __will be considered plagiarism__ and be reported to the student disciplinary committee.

In [1]:
# you may ignore this cell
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion
from sklearn.impute import MissingIndicator
from sklearn.pipeline import Pipeline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from os import path

if os.name == 'nt':
    ANSWER_KEY_FILE_NAME = "answer_key(win).p"
elif os.name == 'posix':
    ANSWER_KEY_FILE_NAME = "answer_key(unix).p"
else:
    raise Exception('The code was not tested on',os.name)

GENERATE_ANSWER_KEY=False

if GENERATE_ANSWER_KEY: 
    ANSWER_KEY = {} 
else:        
    with open(ANSWER_KEY_FILE_NAME, "rb") as f:
        ANSWER_KEY = pickle.load( f )           

In [2]:
# example for using answer key
if GENERATE_ANSWER_KEY==False: 
    print(ANSWER_KEY['log_reg_coefficients'])

In [3]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


# Beginning of homework 5

In previous homework assignments you created and evaluated models based on the HELOC dataset. Let's reload the data.

In [4]:
PREPROCESSED_DATA_FILE_NAME = "preprocessed_data.p"
with open(PREPROCESSED_DATA_FILE_NAME, "rb") as f:
    PREPROCESSED_DATA = pickle.load( f )               

X_train_t_tr,  Y_train_t_tr  = PREPROCESSED_DATA['X_train_t_tr'],  PREPROCESSED_DATA['Y_train_t_tr']
X_train_t_val, Y_train_t_val = PREPROCESSED_DATA['X_train_t_val'], PREPROCESSED_DATA['Y_train_t_val']
X_test_t,      Y_test        = PREPROCESSED_DATA['X_test_t'],      PREPROCESSED_DATA['Y_test']
n = len(Y_train_t_tr) + len(Y_train_t_val) + len(Y_test)
column_names = PREPROCESSED_DATA['column_names']

print('Train data:      X_train_t_tr:', X_train_t_tr.shape, ' Y_train_t_tr:', Y_train_t_tr.shape)
print('Validation data: X_train_t_val:', X_train_t_val.shape, 'Y_train_t_val:', Y_train_t_val.shape)
print('Test data:       X_test_t:', X_test_t.shape, '     Y_test:', Y_test.shape)
print('\nThe proportion of train,validation,test data is %.1f:%.1f:%.1f\n'%(len(Y_train_t_tr)/n, len(Y_train_t_val)/n, len(Y_test)/n))
print('column_names:',column_names)

The file `preprocessed_data.p` contains the preprocessed data split into 3 sets: train, validation, and test. The previous code cell loaded the data to memory and assigned it to variables indicating the set and whether the variable refers to features (X) or labels (Y). That is, `X_train_t_tr`, `X_train_t_val`, and `X_test_t` hold the preprocessed train, validation, and test sets, while `Y_train_t_tr`, `Y_train_t_val`, and `Y_test`, hold the corresponding labels. The variable `column_names` holds the list of column names. 


## Part 1: Linear models

Let's use the train set to train a few linear models and compare them.

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier

In [6]:
log_reg = LogisticRegression(max_iter=10000, random_state=0).fit(X_train_t_tr, Y_train_t_tr) # Logistic regression
svc     = SGDClassifier(max_iter=1000, random_state=0).fit(X_train_t_tr, Y_train_t_tr)       # Linear SVM
nb      = BernoulliNB().fit(X_train_t_tr, Y_train_t_tr)                                      # Naive Bayes
lda     = LinearDiscriminantAnalysis().fit(X_train_t_tr, Y_train_t_tr)                       # LDA

In [7]:
print('Train accuracy')
print('Log. Reg. accuracy: %.3f'%log_reg.score(X_train_t_tr, Y_train_t_tr))
print('Linear SVC accuracy: %.3f'%svc.score(X_train_t_tr, Y_train_t_tr))
print('Naive Bayes accuracy: %.3f'%nb.score(X_train_t_tr, Y_train_t_tr))
print('LDA accuracy: %.3f'%lda.score(X_train_t_tr, Y_train_t_tr)) 

print('\nValidation accuracy')
print('Log. Reg. accuracy: %.3f'%log_reg.score(X_train_t_val, Y_train_t_val))
print('Linear SVC accuracy: %.3f'%svc.score(X_train_t_val, Y_train_t_val))
print('Naive Bayes accuracy: %.3f'%nb.score(X_train_t_val, Y_train_t_val))
print('LDA accuracy: %.3f'%lda.score(X_train_t_val, Y_train_t_val)) 

### q1
Initialize the variables `log_reg_coefficients`, `svc_coefficients`, `nb_coefficients`, and `lda_coefficients` so that each holds an array with the linear coefficients followed by the intercept corresponding to each model (that is, the intercept should be stored as the last element of `log_reg_coefficients` and the other arrays).

In [8]:
log_reg_coefficients = "replace this string with your answer"
svc_coefficients = "replace this string with your answer"
nb_coefficients = "replace this string with your answer"
lda_coefficients = "replace this string with your answer"
###
### YOUR CODE HERE
###


In [9]:
# example
ANSWER_KEY['log_reg_coefficients']

In [10]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['log_reg_coefficients'], log_reg_coefficients, rtol=0, atol=0.01).all()), 'testing log_reg_coefficients'
### END TEST

In [11]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['svc_coefficients'], svc_coefficients, rtol=0, atol=0.01).all()), 'testing svc_coefficients'
### END TEST

In [12]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['nb_coefficients'], nb_coefficients, rtol=0, atol=0.01).all()), 'testing nb_coefficients'
### END TEST

In [13]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['lda_coefficients'], lda_coefficients, rtol=0, atol=0.01).all()), 'testing lda_coefficients'
### END TEST

### q2 
Create the dataframe `linear_coefficients` which holds the coefficients of the 4 models. Each column should correspond to a model (`Log. Reg.`,`SVC`,`NB` and `LDA`; use these exact column names). The rows should hold the column names of the HELOC dataset (which can be found in the variable `column_names` which was initialized above), with one additional row for the intercept of each linear function.

In [14]:
linear_coefficients = "replace this string with your answer"
###
### YOUR CODE HERE
###


In [15]:
# expected output
ANSWER_KEY['linear_coefficients'].head()

In [16]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['linear_coefficients'].round(2)
diff = sol.compare(linear_coefficients.round(2), keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing linear_coefficients'
### END TEST 

### q3
Linear classifiers are equivalent up to a multiplication by a constant. Let's divide each classifier by its intercept to see if the models are different. 

Create the dataframe `linear_coefficients_scaled` which holds the linear coefficients scaled by their intercept values.

In [17]:
linear_coefficients_scaled = "replace this string with your answer"
###
### YOUR CODE HERE
###


In [18]:
# expected output
ANSWER_KEY['linear_coefficients_scaled'].head()

In [19]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['linear_coefficients_scaled'].round(2)
diff = sol.compare(linear_coefficients_scaled.round(2), keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing linear_coefficients_scaled'
### END TEST 

By examining `linear_coefficients_scaled`, we see that the resulting linear models are indeed different. We can also see it in the figure below.

In [20]:
ANSWER_KEY['linear_coefficients_scaled'].plot.bar(figsize=(20,5))
plt.title('Comparison of linear coefficents among different linear models');

We also observe that the models are not identical by looking at their predictions. (Note that the models could be quite similar to each other, which is often the case).

In [21]:
fig, axes = plt.subplots(1,4,figsize=(15,4))
axes[0].hist(log_reg.predict(X_train_t_val))
axes[1].hist(svc.predict(X_train_t_val))
axes[2].hist(nb.predict(X_train_t_val))
axes[3].hist(lda.predict(X_train_t_val));
axes[0].set_title('Log. Reg.\n (%d "Bad" predictions)'%sum(log_reg.predict(X_train_t_val)))
axes[1].set_title('SVC\n (%d "Bad" predictions)'%sum(svc.predict(X_train_t_val)))
axes[2].set_title('NB\n (%d "Bad" predictions)'%sum(nb.predict(X_train_t_val)))
axes[3].set_title('LDA\n (%d "Bad" predictions)'%sum(lda.predict(X_train_t_val)));

Next, let us manually compute the predictions that each model makes. To this end, for a given an observation $x$, we will need to multiply each feature of $x$ by its respective coefficient and then add the intercept.

In [22]:
# example of manually computing the prediction of the logistic regression model 
# we will work with the first observation in the data
observation = X_train_t_val.iloc[0] 

# element-wise multiplication by coefficients, summation, and addition of the intercept
sum(observation*log_reg_coefficients[:-1]) + log_reg_coefficients[-1] 

In [23]:
# an equivalent way using np.dot (inner product)
np.dot(X_train_t_val.iloc[0],log_reg_coefficients[:-1]) + log_reg_coefficients[-1] 

In [24]:
log_reg_coefficients.shape

The first 34 values of `log_reg_coefficients` hold the linear coefficients and the last value holds the intercept. Since the linear function returned positive value, the prediction is `1`, otherwise, the prediction would have been `0`. In the next cell, we apply the linear function to the first 4 observations in `X_train_t_val`:

In [25]:
np.dot(X_train_t_val.iloc[:4,:],log_reg_coefficients[:-1]) + log_reg_coefficients[-1] 

We can compare each the result with 0 to see if the predictions are `1` or `0`:

In [26]:
(np.dot(X_train_t_val.iloc[:4,:],log_reg_coefficients[:-1]) + log_reg_coefficients[-1] >0).astype(int)

And this is exactly what we get when we use the logistic regression model prediction function `.predict()`

In [27]:
log_reg.predict(X_train_t_val.iloc[:4,:])

As a sanity check, we compare the predictions on all datasets:

In [28]:
for X in [X_train_t_tr, X_train_t_val, X_test_t]: # compare train, validation and test sets
    print(((np.dot(X,log_reg_coefficients[:-1]) + log_reg_coefficients[-1] >0).astype(int) == log_reg.predict(X)).all())

### q4
Using the function `np.dot` and `np.mean` compute the validation set accuracy of the logistic regression model. Assign the answer to the variable `log_reg_val_accuracy` (you should not use the function `.predict()` or any other sk-learn functions).

In [29]:
log_reg_val_accuracy = "replace this string with your answer"
###
### YOUR CODE HERE
###


In [30]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['log_reg_val_accuracy'], log_reg_val_accuracy, rtol=0, atol=0.01).all()), 'testing log_reg_val_accuracy'
### END TEST

In [31]:
# expected answer
ANSWER_KEY['log_reg_val_accuracy']

Let us now take a closer look at the coefficients of the logistic regression model.

In [32]:
linear_coefficients['Log. Reg.'].plot.bar(figsize=(15,5));

Some of the coefficients are considerably smaller than others. 

In [33]:
linear_coefficients['Log. Reg.']

Does this mean we can ignore them? Let's try.

### q5
The coefficient associated with `ExternalRiskEstimate` is an order of magnitude smaller than some of the other coefficients. Create the pandas series `modified_log_reg` which is identical to `linear_coefficients['Log. Reg.']` with the difference that the coefficient associated with `ExternalRiskEstimate` is set to 0.

In [34]:
modified_log_reg = "replace this string with your answer"
###
### YOUR CODE HERE
###


In [35]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['modified_log_reg'].round(2)
diff = sol.compare(modified_log_reg.round(2), keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing modified_log_reg'
### END TEST 

Compute the prediction accuracy on the validation set of the new model represented by `modified_log_reg` and assign the answer to the variable `accuracy_modified_log_reg`.

In [36]:
accuracy_modified_log_reg = "replace this string with your answer"
###
### YOUR CODE HERE
###


In [37]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['accuracy_modified_log_reg'], accuracy_modified_log_reg, rtol=0, atol=0.01).all()), 'testing accuracy_modified_log_reg'
### END TEST

In [38]:
ANSWER_KEY['accuracy_modified_log_reg']

The new model is not performing well. The reason is that the magnitude of the features is different:

In [39]:
X_train_t_tr.mean()

The average value of `ExternalRiskEstimate` is among the highest. We can compute the average contribution of each feature to the linear function:

In [40]:
(X_train_t_val* log_reg.coef_[0]).mean().abs().sort_values(ascending=False)

We see that on average, `ExternalRiskEstimate` has large impact on the predictions, despite it's relative small coefficient value (shown below)

In [41]:
linear_coefficients['Log. Reg.'].abs().sort_values(ascending=False)

Let's try something different. Let's train another logistic model but this time train it without `ExternalRiskEstimate` which happens to be the first feature. initialize the variables `X_train_t_tr_wo_ExternalRiskEstimate` and `Y_train_t_tr_wo_ExternalRiskEstimate` to hold the train data without the feature `ExternalRiskEstimate` (based on `X_train_t_tr` and `Y_train_t_tr`). Similarly, create the variables 
`X_train_t_val_wo_ExternalRiskEstimate` and `Y_train_t_val_wo_ExternalRiskEstimate` corresponding to the validation set (based on `X_train_t_val` and `Y_train_t_val`). 

Observe that there no changes are made to the labels here.

In [42]:
X_train_t_tr_wo_ExternalRiskEstimate = "replace this string with your answer"
Y_train_t_tr_wo_ExternalRiskEstimate = "replace this string with your answer"
X_train_t_val_wo_ExternalRiskEstimate = "replace this string with your answer"
Y_train_t_val_wo_ExternalRiskEstimate = "replace this string with your answer"

###
### YOUR CODE HERE
###


In [43]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['X_train_t_tr_wo_ExternalRiskEstimate'].round(2)
diff = sol.compare(X_train_t_tr_wo_ExternalRiskEstimate.round(2), keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing X_train_t_tr_wo_ExternalRiskEstimate'
### END TEST 

In [44]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['Y_train_t_tr_wo_ExternalRiskEstimate'].round(2)
diff = sol.compare(Y_train_t_tr_wo_ExternalRiskEstimate.round(2), keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing Y_train_t_tr_wo_ExternalRiskEstimate'
### END TEST 

In [45]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['X_train_t_val_wo_ExternalRiskEstimate'].round(2)
diff = sol.compare(X_train_t_val_wo_ExternalRiskEstimate.round(2), keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing X_train_t_val_wo_ExternalRiskEstimate'
### END TEST 

In [46]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['Y_train_t_val_wo_ExternalRiskEstimate'].round(2)
diff = sol.compare(Y_train_t_val_wo_ExternalRiskEstimate.round(2), keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing Y_train_t_val_wo_ExternalRiskEstimate'
### END TEST 

In [47]:
# Sanity check:
print('dimensions of X_train_t_tr:',X_train_t_tr.shape) 
print('dimensions of X_train_t_tr_wo_ExternalRiskEstimate:',ANSWER_KEY['X_train_t_tr_wo_ExternalRiskEstimate'].shape)

Now let's train and evaluate the model that does not use the feature `ExternalRiskEstimate`.

In [48]:
log_reg_wo_ExternalRiskEstimate = LogisticRegression(max_iter=10000).fit(ANSWER_KEY['X_train_t_tr_wo_ExternalRiskEstimate'], 
                                                                         ANSWER_KEY['Y_train_t_tr_wo_ExternalRiskEstimate']) 

Compute the train and validation accuracies of the new linear model `log_reg_wo_ExternalRiskEstimate` and assign the results to the variables `accuracy_retrained_model_train` and `accuracy_retrained_model_validation`. Create the variable `retrained_log_reg` which is a pandas series whose indexes are similar to that of `modified_log_reg` and whose values are used by the linear classifier `log_reg_wo_ExternalRiskEstimate`. 

Note that while the feature `ExternalRiskEstimate` is not needed for the model `log_reg_wo_ExternalRiskEstimate`, we keep it to make it easier to compare it with other models. The coefficient associated with `ExternalRiskEstimate` should simply be set equal to 0.

In [49]:
accuracy_retrained_model_train = "replace this string with your answer"
accuracy_retrained_model_validation = "replace this string with your answer"
retrained_log_reg = "replace this string with your answer"

###
### YOUR CODE HERE
###


In [50]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['accuracy_retrained_model_train'], accuracy_retrained_model_train, rtol=0, atol=0.01).all()), 'testing accuracy_retrained_model_train'
### END TEST

In [51]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['accuracy_retrained_model_validation'], accuracy_retrained_model_validation, rtol=0, atol=0.01).all()), 'testing accuracy_retrained_model_validation'
### END TEST

In [52]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['retrained_log_reg'].round(2)
diff = sol.compare(retrained_log_reg.round(2), keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing retrained_log_reg'
### END TEST 

Let's compare accuracies and linear coefficients

In [53]:
print('Log. Reg. retrained model train accuracy: %.3f'%accuracy_retrained_model_train)
print('Log. Reg. retrained model validation accuracy: %.3f\n'%accuracy_retrained_model_validation)

print('Log. Reg. original model train accuracy: %.3f'%log_reg.score(X_train_t_tr, Y_train_t_tr))
print('Log. Reg. original model validation accuracy: %.3f'%log_reg.score(X_train_t_val, Y_train_t_val))

The performance (in terms of accuracy) is quite similar. Let's compare the coefficients:

In [54]:
pd.merge(linear_coefficients['Log. Reg.'],ANSWER_KEY['retrained_log_reg'], left_index=True, right_index=True).plot.bar(figsize=(20,8));

In [55]:
# simliar plot only without the intercept
pd.merge(linear_coefficients['Log. Reg.'],ANSWER_KEY['retrained_log_reg'], left_index=True, right_index=True).iloc[:-1,:].plot.bar(figsize=(20,8));

The coefficients are somewhat different. It appears that training a logistic regression model without the feature `ExternalRiskEstimate` results in a redistribution of the coefficients.

It turns out that in practice, there are often multiple models whose performance is very similar. This, for example, could result from a correlation between features which allows using different subsets of features. It raises the question of which model to use? 

Typical practical answers are:
* use the simplest one. This is known as Occam's razor, and can be obtained by trial and error, regularization, stepwise selection (see Chapter 6 in Introduction in Statistical Learning), or integer programming based models (e.g., SLIM and RiskSLIM).
* look carefully at the features used by each model and see which make more sense in the context of the specific prediction problem.

When debating between different models (trees vs. linear), one should consider the perspective of the user of the model and think about which one would be better suited for him or her for informing decisions. 

__Estimating probabilities__

Finally, let's briefly explore some functionalities of probabilistic models. The logistic regression model not only makes predictions (1 vs. 0) but also estimates the probability that an observation is predicted as 1 or 0. 

In sk-learn, the function `predict` returns a discrete prediction while the function `predict_proba` returns probability.

In [56]:
# make predictions about observations 0 and 1 in the validation set
log_reg.predict(X_train_t_val.iloc[[0,1],:]) 

In [57]:
#  compute probabilities corresponding to observation 0 and 1 in the validation set
print(log_reg.predict_proba(X_train_t_val.iloc[[0,1],:]))

The probability that the label associated with observation #0 is 1, is equal to 0.82, which is greater than 0.5 and therefore the prediction is 1. 

Let's look at the distribution of predictions. We will bucket observations according to the probabilities estimated by the model, and compute the accuracy in each bucket.

### q6

Create the dataframe `df_probs` which holds the following information about the validation set `X_train_t_val`:
1. `Prob(0)` - the probability that the model `log_reg` estimates for the label 0
2. `Prob(1)` - the probability that the model `log_reg` estimates for the label 1
3. `Prediction` - the prediction
4. `Label` - the true label
5. `Correct prediction` - is the prediction correct

An example of the output is provided below.

__Use may the function `log_reg.predict_proba` to compute probabilities.__

In [58]:
df_probs = "replace this string with your answer"

###
### YOUR CODE HERE
###


In [59]:
# expected output
ANSWER_KEY['df_probs'].head()

In [115]:
### BEGIN TEST (DO NOT REMOVE CELL) (required handling rounding)
sol = ANSWER_KEY['df_probs']
diff = sol.loc[:,"Correct prediction"].compare(df_probs.loc[:,"Correct prediction"], keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing "Correct prediction" column of df_probs'

diff = pd.Series({"Prob(0)" : True,
                  "Prob(1)" : True,
                  "Prediction" : True,
                  "Label" : True}).compare(((ANSWER_KEY['df_probs'].iloc[:,:-1] - df_probs.iloc[:,:-1]).abs() <= 0.01).all(), keep_equal=False)
assert(len(diff)==0), 'testing remaining columns of df_probs'
### END TEST 

In [61]:
fig,axes = plt.subplots(1,2,figsize=(20,8))
axes[0].hist(ANSWER_KEY['df_probs']['Prob(1)'], bins=50)
axes[0].set_title('Distribution of predictions')
ANSWER_KEY['df_probs'].groupby('Prob(1)').mean()['Correct prediction'].plot(ax=axes[1], title='% of correct predictions as function of Prob(1)');
axes[1].axhline(ANSWER_KEY['df_probs']['Correct prediction'].mean());

The figure shows that in many cases (when the probability is sufficiently small or large; see the histogram on the left), we are more confident about the model predictions (that's when the percentage of correct predictions is high). 

However, there are also many observations for which the probability is close to 0.5. In these cases we are less confident about our predictions (on the right figure we see that prediction accuracy drops when the probability is around 0.5). 

Intuitively, this is because observations for which the predicted probability is close to 0.5 are close to the decision boundary, an area that is typically more difficult to classify.

## Part 2: Tree-based models

We now switch to working with tree-based models. Let's train a decision tree.

In [62]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from IPython.display import Image 
from IPython.display import IFrame
import pydot_ng as pydot 

### q7 Decision trees

In [63]:
clf_tree = DecisionTreeClassifier(max_depth=1, random_state=0).fit(X_train_t_tr, Y_train_t_tr)

Compute the train and validation accuracies and assign them to the variables `train_accuracy_DT` and `val_accuracy_DT`.

In [64]:
train_accuracy_DT = "replace this string with your answer"
val_accuracy_DT = "replace this string with your answer"
###
### YOUR CODE HERE
###


In [65]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['train_accuracy_DT'], train_accuracy_DT, rtol=0, atol=0.01).all()), 'testing train_accuracy_DT'
### END TEST

In [66]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['val_accuracy_DT'], val_accuracy_DT, rtol=0, atol=0.01).all()), 'testing val_accuracy_DT'
### END TEST

In [67]:
print('Train accuracy (DT): %.3f'%ANSWER_KEY['train_accuracy_DT'])
print('Validation accuracy (DT): %.3f'%ANSWER_KEY['val_accuracy_DT'])

Let's visualize the tree

In [68]:
with open("tree_stump.dot", 'w') as f:                   
    # export visualization of model to a .dot file
    tree.export_graphviz(clf_tree, out_file=f, feature_names=column_names, filled=True, label='all')     
    
pydot.graph_from_dot_file('tree_stump.dot').write_pdf('tree_stump.pdf') # convert .dot to .pdf
IFrame('tree_stump.pdf', width=400, height=300)                         # display pdf in jupyter

The model is surprisingly accurate considering the fact that it is a decision stump (decision tree of depth 1), and that the accuracy is close to other models we previously trained on this dataset.

Let's see what happens when we change the depth of the tree. Initialize the variable `tree_accuracy` to a dataframe whose indexes run from 1 to 12, with two columns, one for `Train accuracy` and one for `Validation accuracy`. Use the default parameters of `DecisionTreeClassifier` except for the depth and setting `random_state=0`.

In [69]:
tree_accuracy = "replace this string with your answer"

###
### YOUR CODE HERE
###


In [70]:
# expected output
ANSWER_KEY['tree_accuracy'].head()

In [71]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['tree_accuracy'].round(2)
diff = sol.compare(tree_accuracy.round(2), keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing tree_accuracy'
### END TEST 

In [72]:
ANSWER_KEY['tree_accuracy'].plot();

Plotting the train and validation accuracies as a function of the tree depth, we observe that there is no improvement the the validation accuracy (in fact there is a deterioration at some point), while the train accuracy keeps improving. This is an indication of overfitting. This is not surprising with decision trees which can perfectly classify any dataset with sufficiently deep trees (as long as there are no two observations in the dataset with identical features and opposite predictions).

Let's try to tune the hyperparameters of the model to improve its performance.  

In [73]:
IFrame('https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html', width=950, height=300)

In [74]:
param_grid = [{'max_depth':range(1,13),
               'min_samples_leaf':[10,20,100],
               'max_leaf_nodes':[2,4,6,20,100,10000]}]

grid_search = GridSearchCV(DecisionTreeClassifier(random_state=0), 
                           param_grid, 
                           cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0), 
                           scoring='accuracy')

grid_search.fit(X_train_t_tr, Y_train_t_tr)

Read the documentation to see what each of the hyperparameters specified above does.

Create the dataframe `tree_accuracy_grid` based on the results of the grid search, which are stored in the variable `grid_search`. The dataframe should have a row per each configuration, and columns for the hyper parameters `max_depth`, `max_leaf_nodes`, `min_samples_leaf`, and `Accuracy`.

In [75]:
tree_accuracy_grid = "replace this string with your answer"
###
### YOUR CODE HERE
###


In [76]:
# expected output
ANSWER_KEY['tree_accuracy_grid'].head()

In [77]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['tree_accuracy_grid'].round(2)
diff = sol.compare(tree_accuracy_grid.round(2), keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing tree_accuracy_grid'
### END TEST 

In [78]:
fig, axes = plt.subplots(1,2,figsize=(20,5))
plt.suptitle('Decision trees: CV Accuracy vs. depth \n(Distribution over hyperparameters that share the same tree depth)')
sns.boxplot(x="max_depth", y='Accuracy', data=ANSWER_KEY['tree_accuracy_grid'], ax=axes[0]);
tree_accuracy.plot(ax=axes[1]);

The figure above shows the cross validation accuracy as a function of the maximal depth (since we're exploring many configuration, for each depth we have several configurations whose accuracies can be shown as a distribution). The right figure is identical to the figure presented earlier. Observe that the y-axis scale is different.

We see that with the right tuning, we can mitigate the overfitting, which are due to the increasing depth. However, it appears that we do not improve significantly the performance and we may simply select the decision stump.

### q8 Ensemble methods - Random forests

Instead of creating a single increasingly more complex trees, let's to create an ensemble of trees. Compute the train and validation accuracies of the Random Forest model below and assign them to the variables `train_accuracy_RF` and `val_accuracy_RF`.

In [79]:
rf = RandomForestClassifier(max_depth=1, random_state=0).fit(X_train_t_tr, Y_train_t_tr)

In [80]:
train_accuracy_RF = "replace this string with your answer"
val_accuracy_RF = "replace this string with your answer"
###
### YOUR CODE HERE
###


In [81]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['train_accuracy_RF'], train_accuracy_RF, rtol=0, atol=0.01).all()), 'testing train_accuracy_RF'
### END TEST

In [82]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['val_accuracy_RF'], val_accuracy_RF, rtol=0, atol=0.01).all()), 'testing val_accuracy_RF'
### END TEST

In [83]:
print('Train accuracy (RF): %.3f'%ANSWER_KEY['train_accuracy_RF'])
print('Validation accuracy (RF): %.3f'%ANSWER_KEY['val_accuracy_RF'])

We gain a roughly 1% improvement comparing to the decision stump. Let's try to improve the model by performing hyper-parameter tuning. Read the documentation to see what each of the hyperparameters specified above does (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

Note: expect the running time to be longer here. We train `3x2x3x4` RF models, each consisting of 10-50 decision trees.


In [84]:
param_grid = [{'n_estimators':[10,20,50],
               'max_depth':range(1,8),
               'min_samples_leaf':[10,20,100],
               'max_leaf_nodes':[2,4,8,16]}]

grid_search = GridSearchCV(RandomForestClassifier(random_state=0), 
                           param_grid, 
                           cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0), 
                           scoring='accuracy')

grid_search.fit(X_train_t_tr, Y_train_t_tr)

Create the dataframe `rf_accuracy_grid` based on the results of the grid search, which are stored in the variable `grid_search`. The dataframe should have a row per each configuration, and columns for the hyper parameters `n_estimators`, `max_depth`, `max_leaf_nodes`, `min_samples_leaf`, and `Accuracy`.

In [85]:
rf_accuracy_grid = "replace this string with your answer"
###
### YOUR CODE HERE
###


In [86]:
# expected output
ANSWER_KEY['rf_accuracy_grid'].head()

In [87]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['rf_accuracy_grid']
diff = sol.compare(rf_accuracy_grid, keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing rf_accuracy_grid'
### END TEST 

In [88]:
fig, axes = plt.subplots(1,2,figsize=(20,5))
plt.suptitle('CV Accuracy vs. depth: DT (left) and RF (right)\n(Distribution over hyperparameters that share tree depth)')
sns.boxplot(x="max_depth", y='Accuracy', data=ANSWER_KEY['tree_accuracy_grid'], ax=axes[0]);
sns.boxplot(x="max_depth", y='Accuracy', data=ANSWER_KEY['rf_accuracy_grid'], ax=axes[1]);
axes[0].set_ylim(0.65,0.74);axes[1].set_ylim(0.65,0.74);

We see in the figure above that with some hyperparameter tuning we can further improve the accuracy of the RF model.

In [89]:
# the best configuration
grid_search.best_params_

In [90]:
# a model trained using the best configuration on all folds
best_RF_model = grid_search.best_estimator_
best_RF_model

Compute the train and validation accuracies of the optimized Random Forest `best_RF_model` and assign them to the variables `train_accuracy_RF_opt` and `val_accuracy_RF_opt`.

In [91]:
train_accuracy_RF_opt = "replace this string with your answer"
val_accuracy_RF_opt = "replace this string with your answer"
###
### YOUR CODE HERE
###


In [92]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['train_accuracy_RF_opt'], train_accuracy_RF_opt, rtol=0, atol=0.01).all()), 'testing train_accuracy_RF_opt'
### END TEST

In [93]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['val_accuracy_RF_opt'], val_accuracy_RF_opt, rtol=0, atol=0.01).all()), 'testing val_accuracy_RF_opt'
### END TEST

In [94]:
print('Train accuracy (RF optimized): %.3f'%ANSWER_KEY['train_accuracy_RF_opt'])
print('Validation accuracy (RF optimized): %.3f'%ANSWER_KEY['val_accuracy_RF_opt'])

An issue that arises when working with ensemble models is that now our model averages 10 trees (n_estimators=10 in the best configuration), each of a potential depth of 7 (that is, with up to $2^7=128$ leaf nodes). This makes it difficult to understand how the model works. One approach that is used in practice is to compute the _feature importance_ by computing the overall improvement in the model that can be attributed to splitting using each feature. 

In [95]:
# plot feature importance
pd.Series(data=grid_search.best_estimator_.feature_importances_, 
          index=column_names).sort_values().plot.bar(figsize=(15,5), 
                                                     title='Feature importance of best RF model');

It seems that our RF model still heavily relies on the feature `ExternalRiskEstimate`, but now it also utilizes other features.

Let's see how the CV accuracy is affected by the number of trees in the ensemble. Initialize the variable `rf_accuracy` to a dataframe whose indexes run from 10 to 190 with increments of 10, with two columns, one for `Train accuracy` and one for `Validation accuracy`. You should create the RF classifiers as follows: `RandomForestClassifier(n_estimators=?, random_state=0, max_depth=1)` where ? is replaced by the ensemble size.

In [96]:
rf_accuracy = "replace this string with your answer"

###
### YOUR CODE HERE
###


In [97]:
ANSWER_KEY['rf_accuracy'].head()

In [98]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['rf_accuracy'].round(2)
diff = sol.compare(rf_accuracy.round(2), keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing rf_accuracy'
### END TEST 

In [99]:
ANSWER_KEY['rf_accuracy'].plot();

We observe that the performance of the ensemble tends to initially improve and then stabilize as the ensemble grows. 

### q9  Ensemble methods - Boosting

Let's to create another ensemble of trees, this time using Boosting. Compute the train and validation accuracies of the Boosting model below and assign them to the variables `train_accuracy_boosting` and `val_accuracy_boosting`.

In [100]:
clf_boosting = AdaBoostClassifier(random_state=0).fit(X_train_t_tr, Y_train_t_tr)

In [101]:
train_accuracy_boosting = "replace this string with your answer"
val_accuracy_boosting = "replace this string with your answer"
###
### YOUR CODE HERE
###


In [102]:
    ### BEGIN TEST
assert(np.isclose(ANSWER_KEY['train_accuracy_boosting'], train_accuracy_boosting, rtol=0, atol=0.01).all()), 'testing train_accuracy_boosting'
### END TEST

In [103]:
### BEGIN TEST
assert(np.isclose(ANSWER_KEY['val_accuracy_boosting'], val_accuracy_boosting, rtol=0, atol=0.01).all()), 'testing val_accuracy_boosting'
### END TEST

In [104]:
print('Train accuracy (boosting): %.3f'%ANSWER_KEY['train_accuracy_boosting'])
print('Validation accuracy (boosting): %.3f'%ANSWER_KEY['val_accuracy_boosting'])
print('\nTrain accuracy (RF optimized): %.3f'%ANSWER_KEY['train_accuracy_RF_opt'])
print('Validation accuracy (RF optimized): %.3f'%ANSWER_KEY['val_accuracy_RF_opt'])
print('\nTrain accuracy (DT): %.3f'%ANSWER_KEY['train_accuracy_DT'])
print('Validation accuracy (DT): %.3f'%ANSWER_KEY['val_accuracy_DT'])

Using the default hyperparameters, we created an ensemble model that outperforms the previous models. This will not always be the case, and one should not conclude that Boosting is always superior to Random Forests. 

There are several important hyperparameters for tuning the Boosting algorithm. One is the `base_estimator` which refers to the base model that we repeatedly train on (slight variations of) the data. The other two are `n_estimators` and `learning_rate` which go together. The first controlling the size of the ensemble (e.g., number of trees), and the second is the "step size", intuitively indicating the importance we give to each model. The smaller the learning rate, the more iterations we may need to reach the optimal ensemble. 

Let's illustrate the effect of using a small learning rate.

Initialize the variable `boosting_accuracy_learning_rate_0_01` to a dataframe whose indexes run from 10 to 190 with increments of 10, with two columns, one for `Train accuracy` and one for `Validation accuracy`. Create an  AdaBoost classifier as follows: `AdaBoostClassifier(n_estimators=?, random_state=0, learning_rate=0.01)` where ? should be replaced by the ensemble size.

In [105]:
boosting_accuracy_learning_rate_0_01 = "replace this string with your answer"

###
### YOUR CODE HERE
###


In [106]:
ANSWER_KEY['boosting_accuracy_learning_rate_0_01'].head()

In [107]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['boosting_accuracy_learning_rate_0_01'].round(2)
diff = sol.compare(boosting_accuracy_learning_rate_0_01.round(2), keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing boosting_accuracy_learning_rate_0_01'
### END TEST 

In [108]:
ANSWER_KEY['boosting_accuracy_learning_rate_0_01'].plot();

Even though we trained an ensemble with 190 trees, the learning rate is small (0.01) and therefore we will need much larger ensembles to obtain better models (assuming we keep the same learning rate).

Let's try a higher learning rate and train a larger ensemble. Initialize the variable `boosting_accuracy_learning_rate_1` to a dataframe whose indexes run from 10 to 390 with increments of 10, with two columns, one for `Train accuracy` and one for `Validation accuracy`. Create an  AdaBoost classifier as follows: `AdaBoostClassifier(n_estimators=?, random_state=0, learning_rate=1)` where ? should be replaced by the ensemble size.

In [109]:
boosting_accuracy_learning_rate_1 = "replace this string with your answer"

###
### YOUR CODE HERE
###


In [110]:
ANSWER_KEY['boosting_accuracy_learning_rate_1'].head()

In [111]:
### BEGIN TEST (DO NOT REMOVE CELL)
sol = ANSWER_KEY['boosting_accuracy_learning_rate_1'].round(2)
diff = sol.compare(boosting_accuracy_learning_rate_1.round(2), keep_equal=False, align_axis=0)
assert(len(diff)==0), 'testing boosting_accuracy_learning_rate_1'
### END TEST 

In [112]:
ANSWER_KEY['boosting_accuracy_learning_rate_1'].plot();

Now that the learning rate is sufficiently large, we achieve the optimal ensemble size using approximately 60 trees. Beyond that point, we start to overfit: the training accuracy keeps improving while the validation accuracy worsens as the ensemble grows.  

Note that simliarly to linear models where there are many linear models that work differently, there are also many variants of the Boosting algorithm. See, for example, [sk-learn](https://scikit-learn.org/stable/modules/ensemble.html), [XGBoost](https://xgboost.readthedocs.io/en/latest/), and [LightGBM](https://github.com/Microsoft/LightGBM).

# Summary


In this homework assignment, we experimented with linear and tree-based models and their key hyper-parameters to observe how they affect the resulting models. The optional reading and assignments cover additional material about the models used in this assignment.

In [113]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
